In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from os.path import join
import sys
sys.path.append("../")
sys.path.append("../model")

## 경고 무시
import warnings
warnings.filterwarnings('ignore')

## 시각화 툴
import matplotlib.pyplot as plt
import seaborn as sns

## encoding 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras import regularizers

## vocabulary
import tensorflow_datasets as tfds

## tokenizer 
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

## preprocessor
from custom_preprocessor import Preprocessor

## model
from transformer_v1 import transformer

from tensorflow.keras import backend as K

# hyper parameter

In [51]:
# 토크나이저 파라미터
VOCAB_SIZE = 10000

# 학습 파라미터
EPOCHS = 1

# 모델 구조 파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

# Data 준비

In [52]:
preprocessor = Preprocessor()
preprocessed_train, preprocessed_test = preprocessor.preprocess()

In [53]:
preprocessed_train["conversation"].apply(lambda x: len(x.split())).max()

## CLASS_NAMES에 '일반 대화'를 포함시킴
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']

# 수동 매핑 설정
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}


# 'class' 열을 수동 매핑 적용하기 전에 문자열로 변환
preprocessed_train['class'] = preprocessed_train['class'].astype(str).map(class_mapping)
labels = preprocessed_train['class']

In [54]:
# BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 특수 토큰 ID 확인
CLS_TOKEN_ID = tokenizer.cls_token_id  # [CLS] 토큰 ID
SEP_TOKEN_ID = tokenizer.sep_token_id  # [SEP] 토큰 ID
UNK_TOKEN_ID = tokenizer.unk_token_id  # [UNK] 토큰 ID
PAD_TOKEN_ID = tokenizer.pad_token_id  # [PAD] 토큰 ID

# 각 문장에 [CLS], [SEP] 추가
def add_special_tokens(tokenized_texts):
    return [[CLS_TOKEN_ID] + tokenizer.convert_tokens_to_ids(tokens) + [SEP_TOKEN_ID] for tokens in tokenized_texts]

## 토크나이징
tokenized_train = [tokenizer.tokenize(con) for con in preprocessed_train['conversation'].tolist()]
tokenized_test = [tokenizer.tokenize(con) for con in preprocessed_test['text'].tolist()]

## [CLS], [SEP] 토큰 추가
token_id_train = add_special_tokens(tokenized_train)
token_id_test = add_special_tokens(tokenized_test)

# 패딩과 트렁케이션 설정
## 문장의 max length를 test 문장의 최대 길이로 설정한다. 
## padding은 앞에서부터 진행
## max길이를 넘어가면 앞에서부터 자른다. -> 한국말은 끝까지 들어야하므로..
token_id_train = pad_sequences(token_id_train, maxlen=191, dtype='long', padding='pre', truncating='pre', value=PAD_TOKEN_ID)
token_id_test = pad_sequences(token_id_test, maxlen=191, dtype='long', padding='pre', truncating='pre', value=PAD_TOKEN_ID)

In [55]:
token_id_train.shape

(4976, 191)

In [56]:
test_size_rate = 0.2
val_size_rate = 0.1

BATCH_SIZE = 64
BUFFER_SIZE = 10000


# 시드값 설정
seed = 1004
X_train, X_test, y_train, y_test = train_test_split(token_id_train, labels, test_size=test_size_rate, stratify=labels, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size_rate, stratify=y_train, random_state=seed)

# 클래스 가중치 계산 (클래스 4 포함)
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

# 각 샘플에 가중치 할당
sample_weights = np.array([class_weights[y] for y in y_train])

# tf.data.Dataset으로 변환 (sample_weight 포함)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weights))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# 데이터셋을 섞고, 배치 처리
train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Model 설정

In [57]:
# 하이퍼파라미터
VOCAB_SIZE = 1000
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    1310208     inputs[0][0]                     
                                                                 enc_padding_mask[0][0]           
__________________________________________________________________________________________________
global_average_pooling1d_2 (Glo (None, 256)          0           encoder[0][0]          

In [58]:
# y_train
y_true_classes = tf.argmax(y_train, axis=-1)
y_train.shape

(3582,)

In [59]:
def loss_function(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=-1)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False)(y_true, y_pred)
    return tf.reduce_mean(loss)

def accuracy(y_true, y_pred):
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

def recall_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5),K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.clip(y_true, 0, 1))
    recall_ratio = true_positives / (possible_positives + K.epsilon())
    return recall_ratio

def precision_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5), K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(y_pred)
    precision_ratio = true_positives / (predicted_positives + K.epsilon())
    return precision_ratio

def f1_m(y_true, y_pred):
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [60]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [61]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[f1_m])

# 학습

In [63]:
EPOCHS = 100
history = model.fit(
    train_dataset,
    validation_data=val_dataset,  # 검증 데이터셋 추가
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/100
56/56 [==============================] - 4s 76ms/step - loss: 1.5956 - f1_m: 0.0000e+00 - val_loss: 1.5826 - val_f1_m: 0.0000e+00
Epoch 2/100
56/56 [==============================] - 4s 68ms/step - loss: 1.5750 - f1_m: 0.0000e+00 - val_loss: 1.5552 - val_f1_m: 0.0000e+00
Epoch 3/100
56/56 [==============================] - 4s 68ms/step - loss: 1.5506 - f1_m: 0.0062 - val_loss: 1.5280 - val_f1_m: 0.0000e+00
Epoch 4/100
56/56 [==============================] - 4s 68ms/step - loss: 1.5120 - f1_m: 0.0599 - val_loss: 1.4916 - val_f1_m: 0.2050
Epoch 5/100
56/56 [==============================] - 4s 68ms/step - loss: 1.4816 - f1_m: 0.2505 - val_loss: 1.4850 - val_f1_m: 0.3906
Epoch 6/100
56/56 [==============================] - 4s 68ms/step - loss: 1.4733 - f1_m: 0.2394 - val_loss: 1.4914 - val_f1_m: 0.4450
Epoch 7/100
56/56 [==============================] - 4s 69ms/step - loss: 1.4757 - f1_m: 0.2506 - val_loss: 1.4353 - val_f1_m: 0.1195
Epoch 8/100
56/56 [=======================

56/56 [==============================] - 4s 70ms/step - loss: 0.9768 - f1_m: 0.6491 - val_loss: 2.3486 - val_f1_m: 0.6531
Epoch 62/100
56/56 [==============================] - 4s 70ms/step - loss: 0.9816 - f1_m: 0.6541 - val_loss: 2.2571 - val_f1_m: 0.7370
Epoch 63/100
56/56 [==============================] - 4s 70ms/step - loss: 0.9566 - f1_m: 0.6601 - val_loss: 2.1128 - val_f1_m: 0.6658
Epoch 64/100
56/56 [==============================] - 4s 70ms/step - loss: 0.9665 - f1_m: 0.6687 - val_loss: 2.3783 - val_f1_m: 0.5552
Epoch 65/100
56/56 [==============================] - 4s 70ms/step - loss: 0.9519 - f1_m: 0.6717 - val_loss: 2.5086 - val_f1_m: 0.6907
Epoch 66/100
56/56 [==============================] - 4s 70ms/step - loss: 0.9589 - f1_m: 0.6769 - val_loss: 2.4960 - val_f1_m: 0.6344
Epoch 67/100
56/56 [==============================] - 4s 70ms/step - loss: 0.9449 - f1_m: 0.6792 - val_loss: 2.2413 - val_f1_m: 0.6245
Epoch 68/100
56/56 [==============================] - 4s 70ms/step -

In [64]:
model.evaluate(test_dataset)

16/16 [==============================] - 0s 29ms/step - loss: 3.1759 - f1_m: 0.7589


[3.175852060317993, 0.7589392066001892]

# submission

In [65]:
result_vec = model(token_id_test)
y_pred = np.argmax(result_vec, axis=1)

In [66]:
y_pred.shape

(500,)

In [67]:
def save_submission(y_pred, user_name, f1_score=None):
    data_path ="/aiffel/aiffel/dlthon_team5/data"
    save_path ="/aiffel/aiffel/dlthon_team5/submission"
    submission_path = join(data_path, 'new_submission.csv')
    submission = pd.read_csv(submission_path)
    submission.columns = ['idx', 'target']
    submission['target'] = y_pred
    submission_csv_path = '{}/submission_{}_f1score_{}.csv'.format(save_path, user_name, f1_score)
    submission.to_csv(submission_csv_path, index=False)
    print('{} saved!'.format(submission_csv_path))

In [69]:
save_submission(y_pred,'phc_v1','75')

/aiffel/aiffel/dlthon_team5/submission/submission_phc_v1_f1score_75.csv saved!
